In [1]:
#Importar librerías necesarias
import pandas as pd
import numpy as np
import cufflinks as cf
import chart_studio.plotly as py
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected = True)
cf.go_offline()

In [2]:
df = pd.read_csv('Unicorn_Companies.csv')   #Importar los datos
df.head()   #Mostrar los primeros 5 elementos

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits
0,Bytedance,$140,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012,$7.44B,IPO,28,8,5
1,SpaceX,$100.3,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,$6.874B,None,29,12,None
2,Stripe,$95,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,$2.901B,Asset,39,12,1
3,Klarna,$45.6,12/12/2011,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005,$3.472B,Acquired,56,13,1
4,Epic Games,$42,10/26/2018,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991,$4.377B,Acquired,25,5,2


# 1) Limpieza del dataset
Verificamos valores nulos en los datos y posteriormente se limpiará columna a columna analizando cada caso


In [3]:
df_clean = df   #Crear un nuevo dataframe para realizar la limpieza

In [4]:
df_clean.isnull().any()   #Verificar si existe algún valor nulo

Company              False
Valuation ($B)       False
Date Joined          False
Country              False
City                 False
Industry             False
Select Inverstors    False
Founded Year         False
Total Raised         False
Financial Stage      False
Investors Count      False
Deal Terms           False
Portfolio Exits      False
dtype: bool

## 1.1) Columna 1: Company

Verifiquemos si no hay compañias repetidas o que compartan el nombre

In [5]:
df_clean['Company'].value_counts()

Bolt                          2
Fabric                        2
YipitData                     1
TalkingData                   1
Unqork                        1
                             ..
Hasura                        1
Five Star Business Finance    1
Sift                          1
JUUL Labs                     1
Snapdeal                      1
Name: Company, Length: 1035, dtype: int64

Tenemos dos nombres repetidos, veamos si se tratan de compañias distintas

In [6]:
df_clean[df_clean['Company']=='Bolt']

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits
37,Bolt,$11,10/8/2021,United States,San Francisco,Fintech,"Activant Capital, Tribe Capital, General Atlantic",2014,$963M,None,37,3,None
63,Bolt,$8.4,5/29/2018,Estonia,Tallinn,Auto & transportation,"Didi Chuxing, Diamler, TMT Investments",2013,$1.971B,None,28,5,None


En este caso, se tratan de compañías distintas por lo que solamente las renombraremos indicando su país de origen

In [7]:
df_clean[(df_clean['Company']=='Bolt') &
         (df_clean['Country']=='United States')] = df_clean[(df_clean['Company']=='Bolt') &
                                                            (df_clean['Country']=='United States')].replace({'Bolt':'Bolt-US'})

In [8]:
df_clean[(df_clean['Company']=='Bolt') &
         (df_clean['Country']=='Estonia')] = df_clean[(df_clean['Company']=='Bolt') & 
                                                      (df_clean['Country']=='Estonia')].replace({'Bolt':'Bolt-EE'})

In [9]:
df_clean[df_clean['Company']=='Fabric']

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits
578,Fabric,$1.5,2/24/2022,United States,Bellevue,E-commerce & direct-to-consumer,"Redpoint Ventures, Norwest Venture Partners, S...",2017,$292.58M,None,11,2,None
976,Fabric,$1,10/26/2021,United States,New York,"Supply chain, logistics, & delivery","Innovation Endeavors, Aleph, Temasek",1999,None,None,1,1,None


En este caso se tratan de dos compañías con mismo nombre, mismo país pero distinta ciudad, procederemos de forma similar

In [10]:
df_clean[(df_clean['Company']=='Fabric') & 
         (df_clean['City']=='Bellevue')] = df_clean[(df_clean['Company']=='Fabric') & 
                                                    (df_clean['City']=='Bellevue')].replace({'Fabric':'Fabric-BV'})
df_clean[(df_clean['Company']=='Fabric') & 
         (df_clean['City']=='New York')] = df_clean[(df_clean['Company']=='Fabric') & 
                                                    (df_clean['City']=='New York')].replace({'Fabric':'Fabric-NY'})

## 1.2) Columna 2: Valuation

In [11]:
df['Valuation ($B)'].describe()    #Información acerca de la columna

count     1037
unique     200
top         $1
freq       244
Name: Valuation ($B), dtype: object

Vemos que la columna hace referencia a un precio, sin embargo el objeto almacenado son `strings` por lo que se transformará a un valor numérico omitiendo también el símbolo del dolar *$*

In [12]:
df_clean['Valuation ($B)'] = df_clean['Valuation ($B)'].replace({'\$':''},
                                                                regex=True)   #Eliminar el símbolo '$'
df_clean['Valuation ($B)'] = df_clean['Valuation ($B)'].astype(float)    #Convertir los valores a punto flotante

In [13]:
df_clean['Valuation ($B)'].describe()

count    1037.000000
mean        3.292083
std         7.309781
min         1.000000
25%         1.080000
50%         1.600000
75%         3.100000
max       140.000000
Name: Valuation ($B), dtype: float64

## 1.3) Columna 3: Date Joined

Aquí convertiremos la fecha de tipo `string` a `DateTime` dándole el formato aaaa/mm/dd

In [14]:
df_clean['Date Joined'] = pd.to_datetime(df_clean['Date Joined'],format ='%m/%d/%Y')
df_clean['Date Joined'] = df_clean['Date Joined'].dt.strftime('%Y-%m-%d')

In [15]:
df_clean['Date Joined']

0       2017-04-07
1       2012-12-01
2       2014-01-23
3       2011-12-12
4       2018-10-26
           ...    
1032    2022-02-22
1033    2022-02-23
1034    2022-02-23
1035    2022-02-23
1036    2022-02-24
Name: Date Joined, Length: 1037, dtype: object

## 1.4 ) Columna 3: Country

In [16]:
df_clean['Country'].value_counts()

United States           536
China                   168
India                    63
United Kingdom           42
Germany                  24
France                   24
Israel                   20
Canada                   19
Brazil                   16
Singapore                12
South Korea              12
Indonesia                 7
Hong Kong                 6
Australia                 6
Japan                     6
Sweden                    6
Netherlands               6
Mexico                    6
Switzerland               5
Ireland                   5
Norway                    4
Turkey                    3
United Arab Emirates      3
Belgium                   3
Spain                     3
Finland                   3
Thailand                  2
Philippines               2
Denmark                   2
Austria                   2
South Africa              2
Chile                     2
Estonia                   2
Vietnam                   2
Colombia                  2
Senegal             

No hay países repetidos indebidamente, no hace falta modificar nada

## 1.5) Columna 6: Industry

Verificamos si existe alguna categoría errónea o repedita

In [17]:
df_clean['Industry'].value_counts()

Fintech                                                               205
Internet software & services                                          192
E-commerce & direct-to-consumer                                       107
Artificial intelligence                                                71
Health                                                                 69
Supply chain, logistics, & delivery                                    57
Other                                                                  56
Cybersecurity                                                          49
Data management & analytics                                            41
Mobile & telecommunications                                            37
Hardware                                                               33
Auto & transportation                                                  29
Edtech                                                                 28
Consumer & retail                     

In [18]:
df_clean[(df_clean['Industry']=='Finttech')]=df_clean[(df_clean['Industry']=='Finttech')].replace({'Finttech':'Fintech'},regex=True) 
#Añadimos la categoría mal escrita a la correcta

En esa columna encontramos una categoría mal escrita que aparecía como una categoría adicional, se ha corregido para añadirla a la categoría correspondiente,

## 1.6) Columna 5: City

In [19]:
cities = [x[0] for x in df_clean.groupby('City')]
# cities

Notamos que existe un error en ciertos registros donde los datos de **Industría** están sobre la ciudad y los **Select Investors** sobre la de Industría

In [20]:
industries = [x[0] for x in df_clean.groupby('Industry')]
# industries

In [21]:
for x in cities:
    for y in industries:
        if x == y:
            df_replace = df_clean[df_clean['City'] == y]
            df_replace['Select Inverstors'] = df_replace['Industry'].values
            df_replace['Industry'] = df_replace['City'].values
            df_replace['City'] = None
            df_clean[df_clean['City'] == y] = df_replace

In [22]:
df_clean[df_clean['City'] == 'Fintech']

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits


In [23]:
df_clean[df_clean['City']=='Artificial intelligence']

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits


In [24]:
df_clean[df_clean['City']=='E-commerce & direct-to-consumer']

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits


In [25]:
df_clean[df_clean['City']=='Fintech']

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits


In [26]:
df_clean[df_clean['City']=='Internet software & services']

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits


## 1.7) Columna 8: Year Founded

Verificamos si es el tipo correcto de dato

In [27]:
df['Founded Year'].value_counts()    #Información acerca de la columna

2015    144
2014    105
2016    101
2012     87
2013     85
2011     76
2017     67
2018     55
None     43
2010     37
2019     34
2009     33
2008     23
2007     23
2020     23
2005     15
2006     13
2000     12
2021      8
2004      8
2001      8
1999      8
2003      7
1998      4
2002      3
1994      2
1995      2
1992      2
1991      1
1993      1
1990      1
1973      1
1996      1
1984      1
1997      1
1919      1
1979      1
Name: Founded Year, dtype: int64

## 1.8) Columna 9: Total Raised

Aquí lo que tenemos que hacer es convertir el objeto a tipo `float` adecuado usando el sufijo de unidad.

In [28]:
df_clean["Total Raised Unity"] = df["Total Raised"].str[-1]

df_clean["Total Raised"] = df_clean["Total Raised"].replace({"\$": "", "B$": "", "M$": "", "None": np.nan, "K$": ""}, regex=True)
df_clean["Total Raised"] = df_clean["Total Raised"].astype(float)

for i, row in df_clean.iterrows():
    if row["Total Raised Unity"] == "B":
        df.loc[i, "Total Raised"] = row["Total Raised"] * 1_000_000_000
    elif row["Total Raised Unity"] == "M":
        df.loc[i, "Total Raised"] = row["Total Raised"] * 1_000_000
    elif row["Total Raised Unity"] == "K":
        df.loc[i, "Total Raised"] = row["Total Raised"] * 1_000

df_clean = df_clean.drop("Total Raised Unity", axis=1)
df_clean.head()

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits
0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012,7.440000e+09,IPO,28,8,5
1,SpaceX,100.3,2012-12-01,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,6.874000e+09,None,29,12,None
2,Stripe,95.0,2014-01-23,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,2.901000e+09,Asset,39,12,1
3,Klarna,45.6,2011-12-12,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005,3.472000e+09,Acquired,56,13,1
4,Epic Games,42.0,2018-10-26,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991,4.377000e+09,Acquired,25,5,2


## 1.9) Columna 10 y 13: Finantial State / Portfolio Exits

Notamos que estas columnas tiene muchos valores faltantes

In [29]:
(df['Financial Stage'] == 'None').sum() / df.shape[0]

0.95274831243973

In [30]:
(df['Portfolio Exits'] == 'None').sum() / df.shape[0]

0.95274831243973

El 95% de esta columna es nula, por lo que podemos omitirla

In [31]:
df_clean = df_clean.drop('Financial Stage', axis=1)
df_clean = df_clean.drop('Portfolio Exits', axis=1)

## DataSet Limpio

In [32]:
df_clean.head()

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Investors Count,Deal Terms
0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012,7.440000e+09,28,8
1,SpaceX,100.3,2012-12-01,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,6.874000e+09,29,12
2,Stripe,95.0,2014-01-23,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,2.901000e+09,39,12
3,Klarna,45.6,2011-12-12,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005,3.472000e+09,56,13
4,Epic Games,42.0,2018-10-26,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991,4.377000e+09,25,5


# Visualización

## Treemap: País, Industria y Compañía - Valor en la bolsa

In [33]:
# Visualización por País, Industria y Compañía
fig = px.treemap(df_clean,path= ["Country","Industry", "Company"],
            values="Valuation ($B)", color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))

## Treemap: Industria, País y Compañía - Total recaudado

In [34]:
# Visualización por País, Industria y Compañía
fig = px.treemap(df_clean,path= ["Industry","Country", "Company"],
            values="Total Raised", color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))

**Insight 1** De estas dos visualizaciones generales notamos que en cuanto a empresas unicornio, EUA es el país lider seguido por China e India. 

**Insight 2** Por otro lado, respecto a las industrias con mayor capital recaudado son en primer lugar las Fintech seguidas por los E-commerce y los servicios y software por internet.

In [35]:
fig = px.histogram()

country_df = df_clean.groupby('Country')
country_max =country_df["Valuation ($B)"].sum().sort_values(ascending=False)[:10]

px.bar(country_max)

In [36]:
fig = px.histogram()

industry_df = df_clean.groupby('Industry')
industry_max =industry_df["Total Raised"].sum().sort_values(ascending=False)[:10]

px.bar(industry_max)